# 03 - TFX Interactive Training Pipeline Execution

The purpose of this notebook is to interactively run the following `TFX` pipeline steps:
1. Receive hyperparameters using `hyperparam_gen` custom Python component.
2. Extract data from BigQuery using `BigQueryExampleGen` component.
3. Validate the raw data using `StatisticsGen` and `ExampleValidator` components.
4. Process the data using `Transform` component.
5. Train a custom model using `Trainer` component.
7. Evaluate and Validate the custom model using `ModelEvaluator` component.
7. Save the blessed to model registry location using `Pusher` component.
8. Upload the model to Vertex AI using `vertex_model_pusher` custom Python component

The custom components are implemented in the [tfx_pipeline/components.py](tfx_pipeline/components) module.

## Setup

### Import libraries

In [ ]:
pip list

In [1]:
import os
import json
import numpy as np
import tfx
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from tensorflow_transform.tf_metadata import schema_utils
import logging

from src.common import features
from src.model_training import data
from src.tfx_pipelines import components
#from tensorflow.keras.models import Sequential

logging.getLogger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')

print("TFX Version:", tfx.__version__)
print("Tensorflow Version:", tf.__version__)

2024-06-27 07:05:39.378395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 07:05:39.378517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 07:05:39.385875: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-27 07:05:39.417769: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 07:05:41.118743: W tensorflow/compiler/tf2

TFX Version: 1.15.1
Tensorflow Version: 2.15.0


### Setup Google Cloud project

In [2]:
PROJECT = 'supply-chain-twin-349311' # Change to your project id.
REGION = 'us-central1' # Change to your region.
BUCKET = 'test-lora' # Change to your bucket name.
SERVICE_ACCOUNT ="db-migration-genai@supply-chain-twin-349311.iam.gserviceaccount.com"

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP projet id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn'exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
PARENT = f"projects/{PROJECT}/locations/{REGION}"
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)
print("Vertex API Parent URI:", PARENT)

Project ID: supply-chain-twin-349311
Region: us-central1
Bucket name: test-lora
Service Account: db-migration-genai@supply-chain-twin-349311.iam.gserviceaccount.com
Vertex API Parent URI: projects/supply-chain-twin-349311/locations/us-central1


### Set configurations

In [3]:
VERSION = 'v09'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'

WORKSPACE = f'gs://{BUCKET}/{DATASET_DISPLAY_NAME}'
RAW_SCHEMA_DIR = 'src/raw_schema'

MLMD_SQLLITE = 'mlmd.sqllite'
ARTIFACT_STORE = os.path.join(WORKSPACE, 'tfx_artifacts_interactive')
MODEL_REGISTRY = os.path.join(WORKSPACE, 'model_registry')
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME)

## Create Interactive Context

In [4]:
REMOVE_ARTIFACTS = True

if tf.io.gfile.exists(ARTIFACT_STORE) and REMOVE_ARTIFACTS:
    print("Removing previous artifacts...")
    tf.io.gfile.rmtree(ARTIFACT_STORE)
    
if tf.io.gfile.exists(MLMD_SQLLITE) and REMOVE_ARTIFACTS:
    print("Deleting previous mlmd.sqllite...")
    tf.io.gfile.rmtree(MLMD_SQLLITE)
    
print(f'Pipeline artifacts directory: {PIPELINE_ROOT}')
print(f'Local metadata SQLlit path: {MLMD_SQLLITE}')

Removing previous artifacts...
Deleting previous mlmd.sqllite...
Pipeline artifacts directory: gs://test-lora/chicago-taxi-tips/tfx_artifacts_interactive/chicago-taxi-tips-classifier-v09-train-pipeline
Local metadata SQLlit path: mlmd.sqllite


In [5]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = MLMD_SQLLITE
connection_config.sqlite.connection_mode = 3 # READWRITE_OPENCREATE
mlmd_store = mlmd.metadata_store.MetadataStore(connection_config)

context = InteractiveContext(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=connection_config
)

## 1. Hyperparameter generation

In [6]:
hyperparams_gen = components.hyperparameters_gen(
    num_epochs=5,
    learning_rate=0.001,
    batch_size=512,
    hidden_units='64,64',
)

context.run(hyperparams_gen, enable_cache=False)

ExecutionResult(
    component_id: hyperparameters_gen
    execution_id: 1
    outputs:
        hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=hyperparameters_gen, output_key=hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [7]:
json.load(
    tf.io.gfile.GFile(
        os.path.join(
            hyperparams_gen.outputs['hyperparameters'].get()[0].uri, 'hyperparameters.json')
    )
)

{'num_epochs': 5,
 'batch_size': 512,
 'learning_rate': 0.001,
 'hidden_units': [64, 64]}

## 2. Data extraction

In [8]:
from src.common import datasource_utils
from tfx.extensions.google_cloud_big_query.example_gen.component import BigQueryExampleGen
from tfx.proto import example_gen_pb2, transform_pb2

### Extract train and eval splits

In [9]:
sql_query = datasource_utils.get_training_source_query(
    PROJECT, REGION, DATASET_DISPLAY_NAME, ml_use='UNASSIGNED', limit=5000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=4),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
        ]
    )
)

train_example_gen = BigQueryExampleGen(query=sql_query, output_config=output_config)

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location={os.path.join(WORKSPACE, 'tmp')}"
]

context.run(
    train_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=False
)

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Extract test split

In [10]:
sql_query = datasource_utils.get_training_source_query(
    PROJECT, REGION, DATASET_DISPLAY_NAME, ml_use='TEST', limit=1000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="test", hash_buckets=1),
        ]
    )
)

test_example_gen = BigQueryExampleGen(query=sql_query, output_config=output_config)

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location={os.path.join(WORKSPACE, 'tmp')}"
]

context.run(
    test_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=False
)

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 3
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Read sample extract tfrecords

In [11]:
train_uri = os.path.join(train_example_gen.outputs['examples'].get()[0].uri, "Split-train/*")
source_raw_schema = tfdv.load_schema_text(os.path.join(RAW_SCHEMA_DIR, 'schema.pbtxt'))
raw_feature_spec = schema_utils.schema_as_feature_spec(source_raw_schema).feature_spec

def _parse_tf_example(tfrecord):
    return tf.io.parse_single_example(tfrecord, raw_feature_spec)

tfrecord_filenames = tf.data.Dataset.list_files(train_uri)
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
dataset = dataset.map(_parse_tf_example)

for raw_features in dataset.shuffle(1000).batch(3).take(1):
    for key in raw_features:
        print(f"{key}: {np.squeeze(raw_features[key], -1)}")
    print("")

dropoff_grid: [b'POINT(-87.7 42)' b'POINT(-87.7 42)' b'POINT(-87.7 42)']
euclidean: [0. 0. 0.]
loc_cross: [b'POINT(-87.7 42)POINT(-87.7 42)' b'POINT(-87.7 42)POINT(-87.7 42)'
 b'POINT(-87.7 42)POINT(-87.7 42)']
payment_type: [b'Cash' b'Cash' b'Cash']
pickup_grid: [b'POINT(-87.7 42)' b'POINT(-87.7 42)' b'POINT(-87.7 42)']
tip_bin: [0 0 0]
trip_day: [15 23 12]
trip_day_of_week: [3 4 7]
trip_hour: [ 6 18 23]
trip_miles: [1.9 0.9 0.7]
trip_month: [1 1 1]
trip_seconds: [423 218 332]



## 3. Data validation

### Import raw schema

In [12]:
from tfx.dsl.components.common.importer import Importer
from tfx.types.standard_artifacts import Schema

# Define the schema importer component
schema_importer = Importer(
    source_uri=RAW_SCHEMA_DIR,
    artifact_type=Schema,
    reimport=False
)

# Run the component in the TFX pipeline context
context.run(schema_importer)



ExecutionResult(
    component_id: Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 18
        uri: "src/raw_schema"
        custom_properties {
          key: "is_external"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.15.1"
          }
        }
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
            _input_trigger: None
        ))

### Generate statistics

In [13]:
from tfx.components import StatisticsGen

# Define the StatisticsGen component
statistics_gen = StatisticsGen(
    examples=train_example_gen.outputs['examples']
)

# Run the component in the TFX pipeline context
context.run(statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [14]:
!rm -r {RAW_SCHEMA_DIR}/.ipynb_checkpoints/

rm: cannot remove 'src/raw_schema/.ipynb_checkpoints/': No such file or directory


### Validate statistics against schema

In [15]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'],
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 6
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [16]:
context.show(example_validator.outputs['anomalies'])

## 4. Data transformation

In [17]:
_transform_module_file = 'src/preprocessing/transformations.py'

transform = tfx.components.Transform(
    examples=train_example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=_transform_module_file,
    splits_config=transform_pb2.SplitsConfig(
        analyze=['train'], transform=['train', 'eval']),
)

context.run(transform, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying etl.py -> build/lib
copying transformations.py -> build/lib


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


installing to /var/tmp/tmpypy2dikf
running install
running install_lib
copying build/lib/etl.py -> /var/tmp/tmpypy2dikf
copying build/lib/transformations.py -> /var/tmp/tmpypy2dikf
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/tmp/tmpypy2dikf/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d-py3.10.egg-info
running install_scripts
creating /var/tmp/tmpypy2dikf/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d.d

ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Read sample transformed tfrecords

In [18]:
transformed_train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, "Split-train/*")
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

tft_output = tft.TFTransformOutput(transform_graph_uri)
transform_feature_spec = tft_output.transformed_feature_spec()

for input_features, target in data.get_dataset(
    transformed_train_uri, transform_feature_spec, batch_size=3).take(1):
    for key in input_features:
        print(f"{key} ({input_features[key].dtype}): {input_features[key].numpy().tolist()}")
    print(f"target: {target.numpy().tolist()}")

dropoff_grid_xf (<dtype: 'int64'>): [0, 0, 0]
euclidean_xf (<dtype: 'float32'>): [0.40534254908561707, -0.12485973536968231, -0.8420690894126892]
loc_cross_xf (<dtype: 'int64'>): [0, 0, 0]
payment_type_xf (<dtype: 'int64'>): [0, 0, 1]
pickup_grid_xf (<dtype: 'int64'>): [0, 0, 0]
trip_day_of_week_xf (<dtype: 'int64'>): [3, 2, 3]
trip_day_xf (<dtype: 'int64'>): [8, 13, 5]
trip_hour_xf (<dtype: 'int64'>): [18, 7, 4]
trip_miles_xf (<dtype: 'float32'>): [-0.27042198181152344, -0.18341393768787384, -0.4139852225780487]
trip_month_xf (<dtype: 'int64'>): [8, 5, 2]
trip_seconds_xf (<dtype: 'float32'>): [-0.15736213326454163, -0.09590243548154831, -0.16111800074577332]
target: [0, 0, 1]


## 5. Model training

In [19]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.experimental import latest_artifacts_resolver
from tfx.dsl.experimental import latest_blessed_model_resolver

### Get the latest model to warm start

In [20]:
latest_model_resolver = Resolver(
    strategy_class=latest_artifacts_resolver.LatestArtifactsResolver,
    latest_model=tfx.types.Channel(type=tfx.types.standard_artifacts.Model)
)

context.run(latest_model_resolver, enable_cache=False)

ExecutionResult(
    component_id: Resolver
    execution_id: 8
    outputs:
        latest_model: OutputChannel(artifact_type=Model, producer_component_id=Resolver, output_key=latest_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Train the model

In [21]:
_train_module_file = 'src/model_training/runner.py'

trainer = tfx.components.Trainer(
    module_file=_train_module_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    base_model=latest_model_resolver.outputs['latest_model'],
    transform_graph=transform.outputs['transform_graph'],
    hyperparameters=hyperparams_gen.outputs['hyperparameters'],
)

context.run(trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying exporter.py -> build/lib
copying task.py -> build/lib
copying trainer.py -> build/lib
copying runner.py -> build/lib
copying model.py -> build/lib
copying data.py -> build/lib
copying defaults.py -> build/lib


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


installing to /var/tmp/tmp6zpij730
running install
running install_lib
copying build/lib/exporter.py -> /var/tmp/tmp6zpij730
copying build/lib/data.py -> /var/tmp/tmp6zpij730
copying build/lib/model.py -> /var/tmp/tmp6zpij730
copying build/lib/runner.py -> /var/tmp/tmp6zpij730
copying build/lib/trainer.py -> /var/tmp/tmp6zpij730
copying build/lib/task.py -> /var/tmp/tmp6zpij730
copying build/lib/defaults.py -> /var/tmp/tmp6zpij730
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/tmp/tmp6zpij730/tfx_user_code_Train

Processing /var/tmp/tmp08tadj0k/tfx_user_code_Trainer-0.0+88cee01a077ea7ce1a9871764b3718ea318b1da2b217190de116232b22159367-py3-none-any.whl
Epoch 1/5
7/7 [==============================] - 4s 339ms/step - loss: 0.6997 - accuracy: 0.8742 - val_loss: 0.6324 - val_accuracy: 0.8828
Epoch 2/5
7/7 [==============================] - 1s 171ms/step - loss: 0.5947 - accuracy: 0.8795 - val_loss: 0.5389 - val_accuracy: 0.8770
Epoch 3/5
7/7 [==============================] - 1s 160ms/step - loss: 0.4998 - accuracy: 0.8803 - val_loss: 0.4284 - val_accuracy: 0.8945
Epoch 4/5
7/7 [==============================] - 1s 167ms/step - loss: 0.4013 - accuracy: 0.8817 - val_loss: 0.3510 - val_accuracy: 0.8770
Epoch 5/5
7/7 [==============================] - 3s 478ms/step - loss: 0.3236 - accuracy: 0.8781 - val_loss: 0.2685 - val_accuracy: 0.8926


2024-06-27 07:14:10.171758: I external/local_tsl/tsl/platform/cloud/gcs_file_system.cc:630] ### RequestUploadSessionStatus: completed = 1, already_uploaded = 0, file = gs://test-lora/chicago-taxi-tips/tfx_artifacts_interactive/chicago-taxi-tips-classifier-v09-train-pipeline/Trainer/model_run/9/validation/events.out.tfevents.1719472442.mlops-v1.1048911.1.v2


ExecutionResult(
    component_id: Trainer
    execution_id: 9
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## 6. Model evaluation

### Get the latest blessed model for model validation.

In [22]:
blessed_model_resolver = Resolver(
    strategy_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=tfx.types.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessing=tfx.types.Channel(type=tfx.types.standard_artifacts.ModelBlessing)
)

context.run(blessed_model_resolver, enable_cache=False)

ExecutionResult(
    component_id: Resolver
    execution_id: 10
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Evaluate and validate the model against the baseline model.

In [23]:
from tfx.components import Evaluator

In [24]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_tf_example',
            label_key=features.TARGET_FEATURE_NAME,
            prediction_key='probabilities')
    ],
    slicing_specs=[
        tfma.SlicingSpec(),
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[   
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(
                    class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.8}),
                        # Change threshold will be ignored if there is no
                        # baseline model resolved from MLMD (first run).
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10}))),
        ])
    ])


evaluator = Evaluator(
    examples=test_example_gen.outputs['examples'],
    example_splits=['test'],
    model=trainer.outputs['model'],
    baseline_model=blessed_model_resolver.outputs['model'],
    eval_config=eval_config,
    schema=schema_importer.outputs['result']
)

context.run(evaluator, enable_cache=False)

ExecutionResult(
    component_id: Evaluator
    execution_id: 11
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [25]:
evaluation_results = evaluator.outputs['evaluation'].get()[0].uri
print("validation_ok:", tfma.load_validation_result(evaluation_results).validation_ok, '\n')

for entry in list(tfma.load_metrics(evaluation_results))[0].metric_keys_and_values:
    value = entry.value.double_value.value
    if value:
        print(entry.key.name, ":", round(entry.value.double_value.value, 3))

validation_ok: True 

example_count : 1000.0
binary_accuracy : 0.888


## 7. Model pushing

In [26]:
exported_model_location = os.path.join(MODEL_REGISTRY, MODEL_DISPLAY_NAME)

push_destination=tfx.proto.pusher_pb2.PushDestination(
    filesystem=tfx.proto.pusher_pb2.PushDestination.Filesystem(
        base_directory=exported_model_location,
    )
)

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=push_destination
)

context.run(pusher, enable_cache=False)

ExecutionResult(
    component_id: Pusher
    execution_id: 12
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## 8. Model Upload to Vertex AI

In [27]:
serving_runtime = 'tf2-cpu.2-7'
serving_image_uri = f"us-docker.pkg.dev/vertex-ai/prediction/{serving_runtime}:latest"

labels = {
    'dataset_name': DATASET_DISPLAY_NAME,
    'pipeline_name': PIPELINE_NAME
}
labels = json.dumps(labels)

vertex_model_uploader = components.vertex_model_uploader(
    project=PROJECT,
    region=REGION,
    model_display_name=MODEL_DISPLAY_NAME,
    pushed_model_location=exported_model_location,
    serving_image_uri=serving_image_uri,
    model_blessing=evaluator.outputs['blessing'],
    explanation_config='',
    labels=labels
)

context.run(vertex_model_uploader, enable_cache=False)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/1049330678395/locations/us-central1/models/188668498744967168/operations/1837539875704799232
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/1049330678395/locations/us-central1/models/188668498744967168@1
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/1049330678395/locations/us-central1/models/188668498744967168@1')


ExecutionResult(
    component_id: vertex_model_uploader
    execution_id: 13
    outputs:
        uploaded_model: OutputChannel(artifact_type=File, producer_component_id=vertex_model_uploader, output_key=uploaded_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [28]:
vertex_model_uploader.outputs['uploaded_model'].get()[0].get_string_custom_property('model_uri')

'projects/1049330678395/locations/us-central1/models/188668498744967168@1'